In [54]:
import pandas as pd 
import plotly.express as px
import plotly.io as pio
import os

import numpy as np 

csv_path = "../csv/"

pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

# for csv_filename in os.listdir("../csv/"):
#     df = pd.read_csv(f"../csv/{csv_filename}")
#     print(csv_filename, df.head())

df = pd.read_csv(csv_path + os.listdir(csv_path)[0])
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


### Product, Order Date, Order Hour, Purchase Address
### Quantity Ordered, Price Each, Sales

In [75]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

def preprocess_data(df):
    df["Quantity Ordered"] = pd.to_numeric(df["Quantity Ordered"], errors="coerce")
    df["Price Each"] = pd.to_numeric(df["Price Each"], errors="coerce")
    df["Sales"] = df["Price Each"] * df["Quantity Ordered"]
    df["Order Value"] = df["Sales"] / df["Quantity Ordered"]
    df["Order Date"] = pd.to_datetime(df["Order Date"], format="%m/%d/%y %H:%M", errors="coerce")
    df["Date"] = df["Order Date"].dt.date
    df["Time"] = df["Order Date"].dt.time
    df["Month"] = df["Order Date"].dt.month
    df["Week"] = df["Order Date"].dt.isocalendar().week
    df["Day"] = df["Order Date"].dt.day
    df["Hour"] = df["Order Date"].dt.hour
    df["City"] = df["Purchase Address"].str.split(",", expand=True)[1]
    return df

def generate_charts(df, group_by, label):
    charts = {}
    group_key = f"grouped_by_{group_by.lower()}"
    label_key = f"for_{label.lower().replace(' ', '_')}"

    quantity = df.groupby(group_by, as_index=False)["Quantity Ordered"].sum()
    fig_quantity = px.bar(quantity, x=group_by, y="Quantity Ordered", title=f"Quantity Ordered {group_key} {label}")
    charts[f"quantity_ordered_{group_key}_{label_key}"] = pio.to_html(fig_quantity, full_html=False)

    sales = df.groupby(group_by, as_index=False)["Sales"].sum()
    fig_sales = px.bar(sales, x=group_by, y="Sales", title=f"Total Sales {group_key} {label}")
    charts[f"total_sales_{group_key}_{label_key}"] = pio.to_html(fig_sales, full_html=False)

    if group_by == "Product":
        price = df.groupby(group_by, as_index=False)["Price Each"].mean()
        fig_price = px.bar(price, x=group_by, y="Price Each", title=f"Average Price {group_key} {label}")
        charts[f"average_price_{group_key}_{label_key}"] = pio.to_html(fig_price, full_html=False)
    else:
        order_value_avg = df.groupby(group_by, as_index=False)["Order Value"].mean()
        fig_avg = px.bar(order_value_avg, x=group_by, y="Order Value", title=f"Average Order Value {group_key} {label}")
        charts[f"average_order_value_{group_key}_{label_key}"] = pio.to_html(fig_avg, full_html=False)

        order_value_total = df.groupby(group_by, as_index=False)["Order Value"].sum()
        fig_total = px.bar(order_value_total, x=group_by, y="Order Value", title=f"Total Order Value {group_key} {label}")
        charts[f"total_order_value_{group_key}_{label_key}"] = pio.to_html(fig_total, full_html=False)

    return charts

def run_analysis(df, group_index=1, specific_products=None):
    group = ["Product", "Date", "Time", "Month", "Week", "Day", "Hour", "City"]
    group_by = group[group_index]
    all_charts = {}

    all_charts.update(generate_charts(df, group_by, "All Products"))

    if specific_products is None:
        specific_products = df["Product"].dropna().unique().tolist()

    for product in specific_products:
        filtered_df = df[df["Product"] == product]
        if filtered_df.empty:
            continue
        all_charts.update(generate_charts(filtered_df, group_by, product))

    return all_charts

def filter_charts_by_keyword(html_charts: dict, keyword: str) -> dict:
    keyword_lower = keyword.lower()
    return {
        key: value for key, value in html_charts.items()
        if keyword_lower in key.lower()
    }

df = preprocess_data(df)
html_charts = run_analysis(df, group_index=1) 

# Example: filter charts by keyword
charging_charts = filter_charts_by_keyword(html_charts, "charging")
list(charging_charts)

['quantity_ordered_grouped_by_date_for_usb-c_charging_cable',
 'total_sales_grouped_by_date_for_usb-c_charging_cable',
 'average_order_value_grouped_by_date_for_usb-c_charging_cable',
 'total_order_value_grouped_by_date_for_usb-c_charging_cable',
 'quantity_ordered_grouped_by_date_for_lightning_charging_cable',
 'total_sales_grouped_by_date_for_lightning_charging_cable',
 'average_order_value_grouped_by_date_for_lightning_charging_cable',
 'total_order_value_grouped_by_date_for_lightning_charging_cable']

In [63]:
# Product, Order Date, Order Hour, Purchase Address
# Quantity Ordered, Price Each, Sales
# "<metric_name>_grouped_by_<grouping_dimension>_for_<product_label>"
